## Merge the spatially aggregated 1-Dimensional and 2-Dimensional datasets

The WRF-Hydro modeling application outputs should have been aggregated to HUC12s. There were two notebooks that performed this opertation, depending on dimension of model output variables. This notebook merges the 1D and 2D datasets together.

In [ ]:
import time
import numpy as np
import xarray as xr

tic = time.time()

In [ ]:
# Input files
#Paths for 2D and 1D aggregated files
in_file1 = r'/path/to/outputs/agg_out/CONUS_HUC12_WB_2D_19791001_20220930_2.nc'
in_file2 = r'/path/to/outputs/CONUS_HUC12_WB_1D_19791001_20220930.nc'

# Output file
out_file = r'/path/to/outputs/agg_out/CONUS_HUC12_WB_combined_19791001_20220930.nc'

# Name the zone coordinate that contains the HUC12 IDs
zone_name = 'WBDHU12'

# Name the time coordinate
time_coord = 'time'

In [ ]:
ds1 = xr.open_dataset(in_file1)
ds2 = xr.open_dataset(in_file2)

In [ ]:
ds1

In [ ]:
# Plot all variables to see ranges of values
vars_to_plot = [variable for variable in ds1.data_vars if variable not in ['avgT2D', 'avgSOILM', 'avgSNEQV', 'total_gridded_area', 'landmask']] 
ds1[vars_to_plot].isel({zone_name:2}).to_array().plot(row='variable')

In [ ]:
ds2

In [ ]:
# Plot all variables to see ranges of values
vars_to_plot = [variable for variable in ds2.data_vars if variable not in ['Area_sqkm', 'totStreamflow']] 
ds2[vars_to_plot].isel({zone_name:2}).to_array().plot(row='variable')

In [ ]:
# Find values in ds2 not in ds1
import numpy as np
vals_1D_only = ds2[zone_name][~np.in1d(ds2[zone_name], ds1[zone_name])]
print('There are {0} values in ds2 that are not in ds1.'.format(vals_1D_only.shape[0]))
      
vals_2D_only = ds1[zone_name][~np.in1d(ds1[zone_name], ds2[zone_name])]
print('There are {0} values in ds1 that are not in ds2.'.format(vals_2D_only.shape[0]))

In [ ]:
ds_out = xr.merge([ds1, ds2])
ds_out

In [ ]:
#ds_out.sel({zone_name:vals_1D_only[0]})['totStreamflow'].plot()
ds_out.sel({zone_name:vals_2D_only[0]})['totPRECIP'].plot()

In [ ]:
%%time
print('  Writing output to {0}'.format(out_file))
ds_out.to_netcdf(out_file, mode='w', format="NETCDF4", compute=True)

In [ ]:
ds_out.close()

In [ ]:
print('Process completed in {0: 3.2f} seconds.'.format(time.time()-tic))